In [ ]:
# Parameters
msgs = "Ran from Airflow at 2022-07-09T04:00:00+00:00!"


In [ ]:
import time
import os
import requests
import urllib.request
import urllib.parse
import json
import re
import collections
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import dateparser
from datetime import datetime
import numpy as np

In [ ]:
ID = "005"
SITE = "rewmi.com"

In [ ]:
categories = ["actualite","faits-divers","economie","people","revue-de-presse","decryptage","sport"]

In [ ]:
def parseDate(date):
    try:
        return datetime.strptime(str(date), '%Y-%m-%dT%H:%M:%S+00:00')
    except:
        return dateparser.parse(date)

In [ ]:
def getArticleDate(HTML):
    try:
        return HTML.find_all("meta", {"property": "article:published_time"})[0]['content']
    except: return ""

In [ ]:
def getWebPage(URL):
    page  = requests.get(URL)
    return BeautifulSoup(page.text, "html.parser")

In [ ]:
def getArticleNbrReader(HTML):
    try:
        h = HTML.find_all("span", {"class": "post-views"})[0]
        return str(h.text).split(" ")[:-1][0]
    except: return ""

In [ ]:
def getArticleTitle(HTML):       
    h = HTML.find_all("h1", {"class": "name post-title entry-title"})
    return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")

In [ ]:
def getArticleContent(HTML):
    h = HTML.find_all("div", {"class": "entry"})[0].find_all("p")
    return " ".join([i.text for i in h])

In [ ]:
def getArticlesList(KEYWORD):
    page_index = 1
    listArt_links = []
    while(True):
        html = getWebPage(f"https://www.rewmi.com/category/{KEYWORD}/page/{page_index}/")
        listArt = html.find_all("article", {"class": "item-list"})
        if len(listArt) == 0: break
        heartbeat(ID)
        listArt_links.append([i.find('a', href=True)['href'] for i in listArt])
        if parseTime(getWebPage(listArt_links[-1][-1]).find_all("meta", {"property": "article:published_time"})[0]['content']).year == 2019:break
        page_index+=1
    return listArt_links

In [ ]:
def getLinks(category, index):
    page = getWebPage(f"https://www.rewmi.com/category/{category}/page/{index}/")
    _links = page.find_all("article", {"class": "item-list"})
    if len(_links) == 0: return []    
    return [i.find('a', href=True)['href'] for i in _links]    


def getNewLinks(category, checkPoint):
    index = 1
    links = []
    while(True):
        _links = getLinks(category, index)
        if _links == []:return links
        if checkPoint[0] in _links or checkPoint[1] in _links:
            try:
                lastIndex = _links.index(checkPoint[0])
            except: lastIndex = _links.index(checkPoint[0])
            if lastIndex == 0:
                return []
            else:
                links.extend(_links)
                return links[:lastIndex]
        else:
            links.extend(_links)
            index+=1
    return links     



def getArticle(link,category):
        try:
            p = getWebPage(link)
            _title      = getArticleTitle(p)
            _article    = getArticleContent(p)
            _date       = getArticleDate(p)
            _author     = ""
            _commments  = ""
            return [1,{'DATE_SCRAPING':str(datetime.today()),
                  'DATE':parseDate(_date),
                  'DATE_ARTICLE':_date,
                  'CATEGORY':category,
                  'URL':link,
                  "AUTEUR":_author,
                  "TITRE":_title,
                  "SITE":SITE,
                  "CONTENU":_article,
                  "COMMENTAIRES":_commments,}]
        except:
            return [0,link]



def updateJson(path, key, value):
    if not os.path.exists(path):
        _data = {key:[]}
    else:
        try:
            jsonFile = open(path, "r",encoding='utf-8')
            _data = json.load(jsonFile)
            jsonFile.close()
        except:
            print("error UpdateJson")
    _data[key].append(value)
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data, default=str))
    jsonFile.close()
    
        
def saveArticle(document):
    Y = document["DATE"].year
    M = document["DATE"].month
    if not os.path.exists(f"/data/notebooks/DB/DATA/{Y}"):
        os.mkdir(f"/data/notebooks/DB/DATA/{Y}") 
    if not os.path.exists(f"/data/notebooks/DB/DATA/{Y}/{M}"):
        os.mkdir(f"/data/notebooks/DB/DATA/{Y}/{M}")
    return updateJson(f"/data/notebooks/DB/DATA/{Y}/{M}/DATA{ID}.json", "DATA", document)


def saveErrors(errList):
    path =  f"/data/notebooks/DB/ERRORLOGS/ERRLOGS{ID}.json"
    Y = datetime.today().year
    M = datetime.today().month
    try:
        jsonFile = open(path, "r",encoding='utf-8')
        _data = json.load(jsonFile)
        jsonFile.close()
    except:
        print("error in saveErrors")
        
    if f"{Y}{M}" not in _data: 
        _data[f"{Y}{M}"] = [errList]
    else:
        _data[f"{Y}{M}"].append(errList)
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data, default=str))
    jsonFile.close()


def checkPoint(category, link=None):
    path = f"/data/notebooks/DB/CHECKPOINT/CP{ID}.json"
    try:
        jsonFile = open(path, "r",encoding='utf-8')
        _data = json.load(jsonFile)
        jsonFile.close()
    except:
        print("ERR")
    if link == None: return [_data[category]["cp1"], _data[category]["cp2"]]
    _data[category]["cp2"] = _data[category]["cp1"]
    _data[category]["cp1"] = link
    _data[category]["time"] = str(datetime.today())
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data))
    jsonFile.close()
    

    
def update():
    error=[]
    for category in categories:
        _checkPoint = checkPoint(category)
        links = getNewLinks(category, _checkPoint)
        i = 0
        for link in links[::-1]:
            i+=1
            status, document = getArticle(link,category)
            if status==0:error.append(document); continue
            saveArticle(document)
            checkPoint(category,link)
    saveErrors(error)

    
    
def initCP():
    path = f"/data/notebooks/DB/CHECKPOINT/CP{ID}.json"
    _data = {}
    for c in categories:
        _data[c] =""
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data))
    jsonFile.close()    

In [ ]:
update()